In [1]:
import numpy as np 
import pandas as pd
import re
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from tqdm import tqdm

from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
#importing movie metadata and keep necessary columns
meta = pd.read_csv("movies_metadata.csv")
meta = meta[['id', 'original_title', 'original_language',
             'revenue', 'vote_average', 'vote_count', 'popularity', 'genres']]
meta = meta.rename(columns={'id':'movieId'})
meta = meta[meta['original_language']== 'en']
meta.head()

,movieId,original_title,original_language,revenue,vote_average,vote_count,popularity,genres
0,862,Toy Story,en,373554033.0,7.7,5415.0,21.9469,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,262797249.0,6.9,2413.0,17.0155,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,0.0,6.5,92.0,11.7129,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,81452156.0,6.1,34.0,3.85949,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,76578911.0,5.7,173.0,8.38752,"[{'id': 35, 'name': 'Comedy'}]"


In [3]:
meta.genres = [list(map(int, re.findall('\d+', x))) for x in meta.genres]
meta.head()

,movieId,original_title,original_language,revenue,vote_average,vote_count,popularity,genres
0,862,Toy Story,en,373554033.0,7.7,5415.0,21.9469,"[16, 35, 10751]"
1,8844,Jumanji,en,262797249.0,6.9,2413.0,17.0155,"[12, 14, 10751]"
2,15602,Grumpier Old Men,en,0.0,6.5,92.0,11.7129,"[10749, 35]"
3,31357,Waiting to Exhale,en,81452156.0,6.1,34.0,3.85949,"[35, 18, 10749]"
4,11862,Father of the Bride Part II,en,76578911.0,5.7,173.0,8.38752,[35]


In [4]:
max_length = len(max(meta.genres, key = len))
print('Max # of Genres: ', max_length)

def padarray(A, size):
    t = size - len(A)
    return np.pad(A, pad_width=(0, t), mode='constant')

meta.genres = [padarray(x, max_length) for x in meta.genres]
meta.head()

Max # of Genres:  8


,movieId,original_title,original_language,revenue,vote_average,vote_count,popularity,genres
0,862,Toy Story,en,373554033.0,7.7,5415.0,21.9469,"[16, 35, 10751, 0, 0, 0, 0, 0]"
1,8844,Jumanji,en,262797249.0,6.9,2413.0,17.0155,"[12, 14, 10751, 0, 0, 0, 0, 0]"
2,15602,Grumpier Old Men,en,0.0,6.5,92.0,11.7129,"[10749, 35, 0, 0, 0, 0, 0, 0]"
3,31357,Waiting to Exhale,en,81452156.0,6.1,34.0,3.85949,"[35, 18, 10749, 0, 0, 0, 0, 0]"
4,11862,Father of the Bride Part II,en,76578911.0,5.7,173.0,8.38752,"[35, 0, 0, 0, 0, 0, 0, 0]"


In [5]:
ref = pd.read_csv("movies_metadata.csv")
ref = ref[['id', 'original_title', 'original_language',
           'revenue', 'vote_average', 'vote_count', 'popularity', 'genres']]
ref = ref.rename(columns={'id':'movieId'})
ref = ref[ref['original_language']== 'en']
ref.genres = [list(map(int, re.findall('\d+', x))) for x in ref.genres]
ref.genres = [padarray(x, max_length) for x in ref.genres]
ref.head()

,movieId,original_title,original_language,revenue,vote_average,vote_count,popularity,genres
0,862,Toy Story,en,373554033.0,7.7,5415.0,21.9469,"[16, 35, 10751, 0, 0, 0, 0, 0]"
1,8844,Jumanji,en,262797249.0,6.9,2413.0,17.0155,"[12, 14, 10751, 0, 0, 0, 0, 0]"
2,15602,Grumpier Old Men,en,0.0,6.5,92.0,11.7129,"[10749, 35, 0, 0, 0, 0, 0, 0]"
3,31357,Waiting to Exhale,en,81452156.0,6.1,34.0,3.85949,"[35, 18, 10749, 0, 0, 0, 0, 0]"
4,11862,Father of the Bride Part II,en,76578911.0,5.7,173.0,8.38752,"[35, 0, 0, 0, 0, 0, 0, 0]"


In [6]:
for n in range(0, max_length):
    meta['genre'+str(n+1)] = meta.genres.apply(lambda x: int(x[n]))

meta.drop('genres', axis=1, inplace=True)
meta.head()

,movieId,original_title,original_language,revenue,vote_average,vote_count,popularity,genre1,genre2,genre3,genre4,genre5,genre6,genre7,genre8
0,862,Toy Story,en,373554033.0,7.7,5415.0,21.9469,16,35,10751,0,0,0,0,0
1,8844,Jumanji,en,262797249.0,6.9,2413.0,17.0155,12,14,10751,0,0,0,0,0
2,15602,Grumpier Old Men,en,0.0,6.5,92.0,11.7129,10749,35,0,0,0,0,0,0
3,31357,Waiting to Exhale,en,81452156.0,6.1,34.0,3.85949,35,18,10749,0,0,0,0,0
4,11862,Father of the Bride Part II,en,76578911.0,5.7,173.0,8.38752,35,0,0,0,0,0,0,0


In [7]:
#importing movie ratings and keep necessary columns
ratings = pd.read_csv("ratings.csv")
ratings = ratings[['userId', 'movieId', 'rating']]

# taking a 2.5MM sample because it can take too long to pivot data later on
ratings = ratings.head(2500000)

#convert data types before merging
meta.movieId = pd.to_numeric(meta.movieId, errors = 'coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors = 'coerce')

#merge the 2 datasets, so that we can have the labels for the movie titles
data= pd.merge(ratings, meta, on = 'movieId', how = 'inner')
data.head()

,userId,movieId,rating,original_title,original_language,revenue,vote_average,vote_count,popularity,genre1,genre2,genre3,genre4,genre5,genre6,genre7,genre8
0,1,858,5.0,Sleepless in Seattle,en,227799884.0,6.5,630.0,10.2349,35,18,10749,0,0,0,0,0
1,3,858,4.0,Sleepless in Seattle,en,227799884.0,6.5,630.0,10.2349,35,18,10749,0,0,0,0,0
2,5,858,5.0,Sleepless in Seattle,en,227799884.0,6.5,630.0,10.2349,35,18,10749,0,0,0,0,0
3,12,858,4.0,Sleepless in Seattle,en,227799884.0,6.5,630.0,10.2349,35,18,10749,0,0,0,0,0
4,20,858,4.5,Sleepless in Seattle,en,227799884.0,6.5,630.0,10.2349,35,18,10749,0,0,0,0,0


In [8]:
#pivot the table so that rows = users and columns = movies and the content is the ratings
matrix= data.pivot_table(index='userId', columns='original_title', values='rating')
matrix.head(10)

original_title,!Women Art Revolution,$5 a Day,'Gator Bait,'R Xmas,'Twas the Night Before Christmas,(A)Sexual,...And the Pursuit of Happiness,10 Items or Less,10 Things I Hate About You,"10,000 BC",...,Æon Flux,Бабник,Грозовые ворота,Дневник его жены,Мой сводный брат Франкенштейн,"Цирк сгорел, и клоуны разбежались",به امید دیدار,مارمولک,რამინი,黑太陽731
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Check to see if the columns are not empty
print('Total ratings score of userId 1: ', matrix.iloc[0].sum())
print('Mean ratings score of userId 1: ', matrix.iloc[0].mean())
print('Ratings Count of userId 1: ', matrix.iloc[0].count())

Total ratings score of userId 1:  30.0
Mean ratings score of userId 1:  4.285714285714286
Ratings Count of userId 1:  7


In [10]:
# Create reference table
data_ref = data
data_ref['target'] = np.where(data_ref.rating < 4, 0, 1)
data_ref['popularity'] = data_ref.popularity.astype(float)

# Load Model
gbc = joblib.load('gbc60000.pkl') 

## Helper Functions

In [11]:
# Pearson Correlation
def pearsonR(s1, s2):
    s1_c = s1-s1.mean()
    s2_c = s2-s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c**2) * np.sum(s2_c**2))

In [12]:
# Create watched list based on userId.
def has_watched(M, userid):
    watched = []
    t = M[M.index==userid].notnull()
    for c in t.columns:
        if t[c].values[0] == True:
            watched.append(c)
    return watched

In [13]:
# Return the score of a recently watched movie
def returnscore(movie, userid, data_ref):
    rs = data_ref.loc[(data_ref.userId == userid) & (data_ref.original_title == movie)].reset_index(drop=True)
    rs.drop(['original_title', 'original_language', 'revenue', 'target'], axis=1, inplace=True)
    rating = rs.iloc[0]['rating']
    if rating < 4:
        s = 0
    else:
        s = 1
    return s, rating

In [14]:
def findcommong(movie1, movie2, ref):
    list1 = ref[ref.original_title == movie1].genres.values
    list1 = list1[0]
    list2 = ref[ref.original_title == movie2].genres.values
    list2 = list2[0]
    common = [i for i in list1 if i in list2 if i != 0]
    return common

In [15]:
def findallcommon(list1, list2, ref):
    all_common = []
    watched_genre = []
    rec_genre = []
    mov_list1 = list1.tolist()
    mov_list2 = list2.tolist()
    genre_matched=[]
    
    for n, title in enumerate(mov_list1):
        
        m1 = ref[ref.original_title == mov_list1[n]].genres.values
        m1 = m1[0]
        m2 = ref[ref.original_title == mov_list2[n]].genres.values
        m2 = m2[0]
        
        watched_genre.append(m1)
        rec_genre.append(m2)
        all_common.append(findcommong(mov_list1[n], mov_list2[n], ref))
    ln = np.concatenate(all_common).ravel().tolist()
    df = pd.DataFrame()
    df['watched_title'] = list1
    df['watched_genre'] = watched_genre
    df['recommended_title'] = list2
    df['recommended_genre'] = rec_genre
    df['in_common_genre'] = all_common
    
    for c in df.in_common_genre:
        if c == []:
            genre_matched.append(0)
        else:
            genre_matched.append(1)
    df['matched_genre'] = genre_matched
    return df, len(ln)

In [16]:
def countcommon(list1):
    ct = 0
    for l in list1:
        if l != []:
            ct += 1
    return ct 

In [17]:
def print_metrics(r, start=0, end=9):
    end = end+1
    print('Accuracy Score: ', accuracy_score(r.target, r.pred))
    print('Average Similarity Score: ', r.similarity_score.mean())
    print('Average Probability Score: ', r.probability_of_pred.mean(), '\n')

    com_hlist, l_hcom = findallcommon(r.watched_title, r.recommended_title, ref)

    print('Out of a total of {} recommendations.'.format(len(r)))
    print("{} recommendations had at least one matching genre.".format(countcommon(com_hlist.in_common_genre)))
    print('{0:.0f}% of watched and recommended genres matched.'.format(countcommon(com_hlist.in_common_genre) / len(r) * 100), '\n')
    print(com_hlist[start:end])

## Transform X function

In [18]:
def getx(movie, userid, data_ref):
    newx = data_ref.loc[data_ref.original_title == movie].reset_index(drop=True)
    newx.drop(['userId', 'rating', 'original_title', 'original_language', 'revenue', 'target'], axis=1, inplace=True)
    newx = newx[:1]

    idx = 0
    new_col = [userid]  
    newx.insert(loc=idx, column='userId', value=new_col)
    return newx

## Main Recommendation Structure

In [19]:
def return_liked_watch(M, userid, data_ref):
    watched=[]
    watch = has_watched(M, userid)
    for mov in watch:
        score, rating = returnscore(mov, userid, data_ref)
        if score == 1:
            watched.append(mov)
    return watched

In [20]:
def get_sim_user(M, n_user, userid):
    
    # A function to make N recommendations based on Pearson Correlation.
    users=[]
    for u in range(0, len(M)):
        if u == userid - 1:
            continue
        
        cor = pearsonR(M.iloc[userid - 1], M.iloc[u])
        if np.isnan(cor):
            continue
        else:
            users.append((u, cor))
    
    # Sort the table of users descending by similarity
    users.sort(key= lambda tup: tup[1], reverse=True)
    usr = pd.DataFrame(users[:n_user], columns=['User', 'Score'])
    
    return usr, M[M.index.isin(usr.User.values)]

In [21]:
def get_recommendation_cf(movie, watched, M2, n_rec, userid, data_ref, ref, model, base=100, matching_genre='default'):
    
    # A function to make N recommendations based on Pearson Correlation.
    reviews=[]
    score, rating = returnscore(movie, userid, data_ref)
    for title in M2.columns:
        if title in watched:
            continue
        
        cor = pearsonR(M2[movie], M2[title])
        if np.isnan(cor):
            continue
        else:
            reviews.append((userid,
                            movie,
                            rating,
                            score,
                            title,
                            cor))
    
    # Sort the table of movies descending by similarity
    rev = pd.DataFrame(reviews, columns=['userId',
                                         'watched_title',
                                         'rating',
                                         'target',
                                         'recommended_title',
                                         'similarity_score'])
    rev.sort_values(by='similarity_score', ascending=False, inplace=True)
    rev.reset_index(drop=True, inplace=True)
    rev = rev[:base]
    
    rec_count=0
    
    uid=[]
    wt=[]
    r=[]
    t=[]
    rt=[]
    ss=[]
    ypred=[]
    proba=[]
    
    for n, title in enumerate(rev.recommended_title):
        X = getx(title, userid, data_ref)
        pred = model.predict(X)
        prob = model.predict_proba(X)
        c = findcommong(rev.watched_title[n], title, ref)
        
        if matching_genre == 'all':
            if int(pred) == 1: 
                if c != []:
                    uid.append(rev.userId[n])
                    wt.append(rev.watched_title[n])
                    r.append(rev.rating[n])
                    t.append(rev.target[n])
                    rt.append(title)
                    ss.append(rev.similarity_score[n])
                    ypred.append(int(pred))
                    proba.append(float(prob[:,int(pred)]))
                    rec_count = rec_count + 1

        elif matching_genre == 'none':
            if int(pred) == 1:
                if c == []:
                    uid.append(rev.userId[n])
                    wt.append(rev.watched_title[n])
                    r.append(rev.rating[n])
                    t.append(rev.target[n])
                    rt.append(title)
                    ss.append(rev.similarity_score[n])
                    ypred.append(int(pred))
                    proba.append(float(prob[:,int(pred)]))
                    rec_count = rec_count + 1
       
        elif matching_genre == 'default':
            if int(pred) == 1:
                uid.append(rev.userId[n])
                wt.append(rev.watched_title[n])
                r.append(rev.rating[n])
                t.append(rev.target[n])
                rt.append(title)
                ss.append(rev.similarity_score[n])
                ypred.append(int(pred))
                proba.append(float(prob[:,int(pred)]))

                rec_count = rec_count + 1
      
        if rec_count == n_rec:
                break
             
    t_df = pd.DataFrame(
    {'userId': uid,
     'watched_title': wt,
     'rating': r,
     'target': t,
     'recommended_title': rt,
     'similarity_score': ss,
     'probability_of_pred': proba,
     'pred': ypred
    })
    
    t_df = t_df[['userId', 'watched_title', 'rating', 'target',
                 'recommended_title', 'similarity_score',
                 'probability_of_pred', 'pred']]
    
    return t_df

In [22]:
def recommend_from_history(matrix, userids, n_user, n_rec, data_ref, ref, model, base=100, matching_genre='default'):
    final_df = pd.DataFrame()
    for userid in tqdm(userids):
        watched = return_liked_watch(matrix, userid, data_ref)
        u, matrix2 = get_sim_user(matrix, n_user, userid)
        for mov in watched:
            r = get_recommendation_cf(mov, watched, matrix2, n_rec, userid, data_ref, ref, model, base=base, matching_genre=matching_genre)
            final_df = pd.concat([final_df, r]).reset_index(drop=True)
    return final_df

In [23]:
comp_user_list = data_ref.userId.unique()
comp_user_list = comp_user_list[:500]
user_list_under10 = []

for u in comp_user_list:
    hw = has_watched(matrix, u)
    if len(hw) < 11:
        user_list_under10.append(u)
        
trunc_user_list = user_list_under10[:50]       
print('Length of list under 10: ', len(user_list_under10))
print('Truncated list under 10: ', trunc_user_list)

Length of list under 10:  72
Truncated list under 10:  [1, 3, 5, 28, 50, 109, 138, 143, 146, 184, 204, 206, 210, 401, 448, 502, 643, 647, 655, 671, 695, 734, 812, 840, 858, 862, 867, 915, 959, 1182, 1202, 1206, 1209, 1303, 1317, 1347, 1377, 1387, 1442, 1446, 1452, 1474, 1533, 1606, 1610, 1632, 1642, 1690, 1715, 1766]


In [24]:
start=datetime.now()
rec_df1 = recommend_from_history(matrix, trunc_user_list, 750, 5, data_ref, ref, gbc, base=200, matching_genre='default')
print('Runtime: ', datetime.now()-start)
rec_df1

100%|██████████| 50/50 [38:57<00:00, 46.75s/it]

Runtime:  0:38:57.663981


,userId,watched_title,rating,target,recommended_title,similarity_score,probability_of_pred,pred
0,1.0,Fools Rush In,4.0,1.0,Music Box,0.360087,0.681440,1.0
1,1.0,Fools Rush In,4.0,1.0,Shakespeare in Love,0.331499,0.934195,1.0
2,1.0,Fools Rush In,4.0,1.0,Hulk,0.328091,0.782082,1.0
3,1.0,Fools Rush In,4.0,1.0,Le Professionnel,0.311649,0.934083,1.0
4,1.0,Fools Rush In,4.0,1.0,Nell,0.307196,0.997465,1.0
5,1.0,License to Wed,4.0,1.0,The Ewok Adventure,0.322721,0.999783,1.0
6,1.0,License to Wed,4.0,1.0,Warlords of the 21st Century,0.264543,0.554028,1.0
7,1.0,License to Wed,4.0,1.0,Minority Report,0.247967,0.907353,1.0
8,1.0,License to Wed,4.0,1.0,Holy Matrimony,0.245785,0.991041,1.0
9,1.0,License to Wed,4.0,1.0,42nd Street,0.245593,0.934039,1.0


In [25]:
print_metrics(rec_df1, start=0, end=9)

Accuracy Score:  1.0
Average Similarity Score:  0.27385202865096814
Average Probability Score:  0.7475733441050753 

Out of a total of 1000 recommendations.
567 recommendations had at least one matching genre.
57% of watched and recommended genres matched. 

    watched_title                   watched_genre  \
0   Fools Rush In  [18, 35, 10749, 0, 0, 0, 0, 0]   
1   Fools Rush In  [18, 35, 10749, 0, 0, 0, 0, 0]   
2   Fools Rush In  [18, 35, 10749, 0, 0, 0, 0, 0]   
3   Fools Rush In  [18, 35, 10749, 0, 0, 0, 0, 0]   
4   Fools Rush In  [18, 35, 10749, 0, 0, 0, 0, 0]   
5  License to Wed       [35, 0, 0, 0, 0, 0, 0, 0]   
6  License to Wed       [35, 0, 0, 0, 0, 0, 0, 0]   
7  License to Wed       [35, 0, 0, 0, 0, 0, 0, 0]   
8  License to Wed       [35, 0, 0, 0, 0, 0, 0, 0]   
9  License to Wed       [35, 0, 0, 0, 0, 0, 0, 0]   

              recommended_title                     recommended_genre  \
0                     Music Box       [80, 18, 10749, 53, 0, 0, 0, 0]   
1          

In [26]:
start=datetime.now()
rec_df2 = recommend_from_history(matrix, trunc_user_list, 5000, 5, data_ref, ref, gbc, base=200, matching_genre='all')
print('Runtime: ', datetime.now()-start)
rec_df2

100%|██████████| 50/50 [49:26<00:00, 59.34s/it]

Runtime:  0:49:26.946812


,userId,watched_title,rating,target,recommended_title,similarity_score,probability_of_pred,pred
0,1.0,Fools Rush In,4.0,1.0,One Night at McCool's,0.199628,0.956518,1.0
1,1.0,Fools Rush In,4.0,1.0,Notes on a Scandal,0.198608,0.999906,1.0
2,1.0,Fools Rush In,4.0,1.0,The Time Machine,0.166665,0.987270,1.0
3,1.0,Fools Rush In,4.0,1.0,Gods and Monsters,0.142228,0.998965,1.0
4,1.0,Fools Rush In,4.0,1.0,My Name Is Bruce,0.134637,0.749316,1.0
5,1.0,License to Wed,4.0,1.0,Beetlejuice,0.158864,0.992277,1.0
6,1.0,License to Wed,4.0,1.0,Don't Make Waves,0.116873,0.996745,1.0
7,1.0,License to Wed,4.0,1.0,Human Nature,0.104257,0.993945,1.0
8,1.0,License to Wed,4.0,1.0,Idle Hands,0.095827,0.963333,1.0
9,1.0,License to Wed,4.0,1.0,Say Anything...,0.090207,0.999424,1.0


In [27]:
print_metrics(rec_df2, start=0, end=9)

Accuracy Score:  1.0
Average Similarity Score:  0.15135642105508285
Average Probability Score:  0.7706117532169954 

Out of a total of 997 recommendations.
997 recommendations had at least one matching genre.
100% of watched and recommended genres matched. 

    watched_title                   watched_genre      recommended_title  \
0   Fools Rush In  [18, 35, 10749, 0, 0, 0, 0, 0]  One Night at McCool's   
1   Fools Rush In  [18, 35, 10749, 0, 0, 0, 0, 0]     Notes on a Scandal   
2   Fools Rush In  [18, 35, 10749, 0, 0, 0, 0, 0]       The Time Machine   
3   Fools Rush In  [18, 35, 10749, 0, 0, 0, 0, 0]      Gods and Monsters   
4   Fools Rush In  [18, 35, 10749, 0, 0, 0, 0, 0]       My Name Is Bruce   
5  License to Wed       [35, 0, 0, 0, 0, 0, 0, 0]            Beetlejuice   
6  License to Wed       [35, 0, 0, 0, 0, 0, 0, 0]       Don't Make Waves   
7  License to Wed       [35, 0, 0, 0, 0, 0, 0, 0]           Human Nature   
8  License to Wed       [35, 0, 0, 0, 0, 0, 0, 0]        